In [1]:
import logging
logging.getLogger().setLevel(logging.INFO)
import mxnet as mx
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\gilbe\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mx.random.seed(1234)
fname = mx.test_utils.download('https://s3.us-east-2.amazonaws.com/mxnet-public/letter_recognition/letter-recognition.data')
data = np.genfromtxt(fname, delimiter=',')[:, 1:]
label = np.array([ord(l.split(',')[0]) - ord('A') for  l in open(fname, 'r')])

INFO:root:letter-recognition.data exists, skipping download


## Iterators

In [3]:
batch_size = 32
ntrain = int(data.shape[0] * 0.8)
train_iter = mx.io.NDArrayIter(data[:ntrain, :], 
                               label[:ntrain], 
                               batch_size,
                               shuffle=True)

val_iter = mx.io.NDArrayIter(data[ntrain:, :], 
                             label[ntrain:], 
                             batch_size)


### Model

In [4]:
net = mx.sym.Variable('data')
net = mx.sym.FullyConnected(net, name='fc1', num_hidden=64)
net = mx.sym.Activation(net, name='relu1', act_type="relu")
net = mx.sym.FullyConnected(net, name='fc2', num_hidden=26)
net = mx.sym.SoftmaxOutput(net, name='softmax')
#mx.viz.plot_network(net)

## Module

In [5]:
mod = mx.mod.Module(symbol=net,
                    context=mx.gpu())

### Intermediate level interface

In [6]:
mod.bind(data_shapes=train_iter.provide_data, 
         label_shapes=train_iter.provide_label)

mod.init_params(initializer=mx.init.Uniform(scale=0.1))

mod.init_optimizer(optimizer='Adam', 
                   optimizer_params=(('learning_rate', 0.1),))

train_iter.reset()
mod.fit(train_iter, 
        eval_data=val_iter, 
        optimizer='Adam', 
        optimizer_params={'learning_rate': 0.1},
        eval_metric='acc', 
        num_epoch=8)

# Use accuracy for the error metric
#metric = mx.metric.create('acc')
#
# train 5 epochs
#for epoch in range(5):
#    train_iter.reset()
#    metric.reset()
#    for batch in train_iter:
#        print(batch.label)
#        mod.forward(batch, is_train=True)
#        mod.update_metric(metric, batch.label)
#        mod.backward()
#       mod.update()
#    print('Epoch %d, Training %s' %(epoch, metric.get()))


C:\Users\gilbe\Anaconda3\lib\site-packages\mxnet\module\base_module.py:464: UserWarning: Parameters already initialized and force_init=False. init_params call ignored.
  allow_missing=allow_missing, force_init=force_init)
INFO:root:Epoch[0] Train-accuracy=0.036125
INFO:root:Epoch[0] Time cost=0.546
INFO:root:Epoch[0] Validation-accuracy=0.039250
INFO:root:Epoch[1] Train-accuracy=0.037000
INFO:root:Epoch[1] Time cost=0.521
INFO:root:Epoch[1] Validation-accuracy=0.039250
INFO:root:Epoch[2] Train-accuracy=0.036750
INFO:root:Epoch[2] Time cost=0.501
INFO:root:Epoch[2] Validation-accuracy=0.039250
INFO:root:Epoch[3] Train-accuracy=0.036750
INFO:root:Epoch[3] Time cost=0.519
INFO:root:Epoch[3] Validation-accuracy=0.039250
INFO:root:Epoch[4] Train-accuracy=0.036750
INFO:root:Epoch[4] Time cost=0.513
INFO:root:Epoch[4] Validation-accuracy=0.039250
INFO:root:Epoch[5] Train-accuracy=0.036750
INFO:root:Epoch[5] Time cost=0.497
INFO:root:Epoch[5] Validation-accuracy=0.039250
INFO:root:Epoch[6] Tra